# Ceci est notebook de test
Tu peux t'inspirer de ce notebook jupyter pour en faire d'autre avec:
 * des exemples
 * des images 
 * des questions
 * ...
 


In [1]:
include("../src/SphereIFSCalib.jl")

using TwoDimensional


┌ Info: Precompiling Zygote [e88e6eb3-aa80-5325-afca-941959d7151f]
└ @ Base loading.jl:1278


### Constantes:

In [2]:

# wavelengths
const λ1 = 987.72e-9# laser 1 
const λ2 = 1123.71e-9# laser 2 
const λ3 = 1309.37e-9# laser 3
const λ4 = 1545.10e-9  # laser 4  
const λ0 = λ1# reference
const λlaser = [λ1,λ2,λ3,λ4]

# model of the LensletModel
bbox = BoundingBox(xmin=1, ymin=1, xmax=10, ymax=10);
lmod = LensletModel(λ0,λlaser,bbox);



### Données & initialisation

In [3]:

data = rand(100,100)
weight = Float64.(rand(100,100).>0.1);


Cinit = rand(Float64,2,4);
a0= rand(Float64,4);
fwhm0= rand(Float64,4);
m = rand(100,100);


### Modèles

In [4]:

likelihood(a,fwhm,C) = GaussianSpotsCost(data,weight,lmod,a,fwhm,C)
cost = likelihood(a0,fwhm0,Cinit)

∇cost = gradient(likelihood,a0,fwhm0,Cinit)

function  g(a::Array{Float64,1},fwhm::Array{Float64,1},C::Array{Float64,2}) 
    return sum(GaussianSpotsModel(lmod,a,fwhm,C))
end

g (generic function with 1 method)